# **Libraries**

In [0]:
import cv2
from PIL import ImageFont, ImageDraw, Image 
import matplotlib.pyplot as plt
import os
import numpy as np
from datetime import datetime as dt
import tensorflow as tf
import string

bold = '\033[1m'
end = '\033[0m'

# **Functions**

In [0]:
def read_data(file_list):
  '''
  read data from tfrecords file
  '''
  file_queue=tf.train.string_input_producer(file_list)
  feature = {'images': tf.FixedLenFeature([], tf.string),
             'labels': tf.FixedLenFeature([], tf.string)}    
  reader = tf.TFRecordReader()  
  _,record=reader.read(file_queue)#read a record
  features = tf.parse_single_example(record, features=feature)
  img = tf.decode_raw(features['images'], tf.uint8)
  label = tf.decode_raw(features['labels'], tf.uint8) 
  return img,label

def minibatch(batch_size, filename, file_count, \
              image_size, max_char, class_count):
  '''
  create minibatch
  '''
  file_list=[os.path.join(filename + \
            '%d.tfrecords' % i) for i in range(1, file_count+1)]  
  img, label=read_data(file_list)
  img = tf.cast(tf.reshape(img,img_size), dtype = tf.float32)
  label = tf.reshape(label[0], [1, max_char])# added [0] as workaround, need to resolve the issue
  label = tf.one_hot(label[0],class_count,axis=1)# added [0] as workaround, need to resolve the issue
  label = tf.reshape(label,tf.shape(label)[1:])
  img_batch,label_batch= tf.train.shuffle_batch([img, label],
                          batch_size,capacity,min_after_dequeue,\
                          num_threads=num_of_threads)
  return img_batch, tf.cast(label_batch, dtype = tf.int64)

def variable(name,shape,initializer,weight_decay = None):
  '''
  create parameter tensor
  '''
  var = tf.get_variable(name, shape, initializer = initializer)
  if weight_decay is not None:
    weight_loss = tf.multiply(tf.nn.l2_loss(var),weight_decay, name="weight_loss")
    tf.add_to_collection('losses', weight_loss)
  return var


#need to customize activation and lrn
def conv_block(block_num,
               input_data,
               weights, 
               weight_initializer=tf.contrib.layers.xavier_initializer(),
               bias_initializer=tf.constant_initializer(0.0),
               conv_op=[1,1,1,1],
               conv_padding='SAME',
               weight_decay=None,
               lrn=True,
               dropout=1.0, 
               activation=True):
  '''
  convolutional block
  '''
  with tf.variable_scope('conv'+ str(block_num), reuse = tf.AUTO_REUSE) as scope:
    input_data = tf.nn.dropout(input_data, dropout)
    kernel = variable('weights', weights, initializer = weight_initializer, weight_decay = weight_decay)
    biases = variable('biases', weights[3], initializer=bias_initializer, weight_decay=None)
    conv = tf.nn.conv2d(input_data, kernel, conv_op, padding=conv_padding)
    pre_activation = tf.nn.bias_add(conv, biases)
    if lrn==True:
      pre_activation = tf.nn.lrn(pre_activation, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm')
    if activation:
      conv_out = tf.nn.relu(pre_activation, name=scope.name)
      return conv_out
    else:
      return pre_activation



def dense_block(block_num,
                input_data,
                neurons,
                weight_initializer=tf.contrib.layers.xavier_initializer(),
                bias_initializer=tf.constant_initializer(0.0),
                weight_decay=None,
                activation=True, 
                dropout=1.0):
  '''
  Fully connected block
  '''
  with tf.variable_scope('dense'+ str(block_num), reuse = tf.AUTO_REUSE) as scope:
    input_data = tf.nn.dropout(input_data, dropout)
    weights = variable('weights', [input_data.shape[1], neurons], \
                       initializer=weight_initializer, weight_decay = weight_decay)
    biases = variable('biases', [1,neurons], initializer = bias_initializer, weight_decay = None)
    dense = tf.matmul(input_data,weights)+biases
    if activation:
      dense=tf.nn.relu(dense, name=scope.name)
    return dense
  
  
  
def loss(logits,labels):
	loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels),name='cross_entropy_loss_mean')
	tf.add_to_collection('losses', loss)
	total_loss=tf.add_n(tf.get_collection('losses'), name='total_loss')
	tf.add_to_collection('losses', total_loss)
	return total_loss


  
def parameter_update(loss, learning_rate):
  '''
  optimization and parameter update using adam optimizer
  '''
  optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
  for var in tf.trainable_variables():
    tf.summary.histogram(var.op.name, var)
  return optimizer



def accuracy_calc(output, label_batch):
  '''
  calculate accuracy
  '''
  correct_prediction = tf.equal(tf.cast(tf.argmax(output, 1),dtype=tf.int32),tf.cast(tf.argmax(label_batch, 1),dtype=tf.int32))
  accuracy=tf.reduce_mean(tf.cast(correct_prediction,"float"))
  return accuracy   

# Models

In [0]:
def inference(image_batch, class_count, weights, dropout=[1,1,1,1],wd=None):
  '''
  Forward propagation
  '''
  i = 0           
  conv_op=[[1,1,1,1],[1,1,1,1],[1,1,1,1], [1,1,1,1]]
  
  conv1 = conv_block(1,image_batch,weights[i], conv_op = conv_op[i], conv_padding='SAME', dropout=dropout[i],weight_decay=wd)
  i=i+1
  pool1=tf.nn.max_pool(conv1, ksize=[1, 4, 4, 1], strides=[1,4,4,1],padding='SAME', name='pool1') #32x32
  
  conv2 = conv_block(2,pool1,weights[i], conv_op = conv_op[i], conv_padding='SAME', dropout=dropout[i],weight_decay=wd)
  i=i+1
  pool2=tf.nn.max_pool(conv2, ksize=[1, 4, 4, 1], strides=[1,4,4,1],padding='SAME', name='pool2') #8x8
  
  conv3 = conv_block(3,pool2,weights[i], conv_op = conv_op[i], conv_padding='SAME', dropout=dropout[i],weight_decay=wd)
  i=i+1
  pool3=tf.nn.max_pool(conv3, ksize=[1, 4, 4, 1], strides=[1,4,4,1],padding='SAME', name='pool3') #2x2
  
  conv4 = conv_block(4,pool3,weights[i], conv_op = conv_op[i], conv_padding='SAME', dropout=dropout[i],weight_decay=wd)
  i=i+1
  pool4=tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1,2,2,1],padding='SAME', name='pool4')#1x1

  flat=tf.reshape(pool4, [tf.shape(image_batch)[0], class_count], name='flat')
		
  return flat

# **Training functions**

In [0]:
def train(folder_path, train_filename, test_filename, 
          train_data_count, file_count,
          weights, dropout, wd,
          img_size, max_char, class_count, 
          batch_size = 32, learning_rate=0.01, epochs=5, 
          restore=False, var_lr = [None,None]):
  
	train_step = train_data_count//batch_size
	start_time = dt.now()
  #build graph
	with tf.Graph().as_default():
		x_train, y_train = minibatch(batch_size, train_filename, file_count, img_size, max_char, class_count)    
		logit_train = inference(x_train, class_count, weights, dropout = dropout, wd = wd)
		cost = loss(logit_train, y_train)
		update=parameter_update(cost,learning_rate)	
		accuracy_train = accuracy_calc(logit_train, y_train)
    
		saver = tf.train.Saver()    
    #start session
		with tf.Session() as sess:
      #initialize the variables
			sess.run(tf.global_variables_initializer())
			sess.run(tf.local_variables_initializer())
			coord = tf.train.Coordinator()
			threads = tf.train.start_queue_runners(coord=coord) 
      
      #restore the variables
			if restore == True:
				loader = tf.train.import_meta_graph(checkpoint_restore +'.meta')
				loader.restore(sess, checkpoint_restore)
        
			#train for given number of epochs
			for e in range(epochs): 
				print(bold + "\nepoch:" + end, e)
				train_epoch_cost = 0
				train_epoch_acc = 0
        
        #train for given number of steps in one epoch
				for s in range(train_step):
					_,train_batch_cost = sess.run([update, cost])	                  
					train_epoch_cost += train_batch_cost/train_step         
				print(bold + "epoch_cost: " + end, train_epoch_cost)
        
        #calculate accuracy of training set
				for i in range(train_step//5):
					train_batch_acc = sess.run(accuracy_train)
					train_epoch_acc = train_epoch_acc + (train_batch_acc/(train_step//5))      
				print(bold + "train epoch accuracy: " + end, train_epoch_acc, "\n")
        
        #afer every lr[0] epoch decrease learning rate by factor of lr[1]
				if var_lr[0] != None:
					if e%var_lr[0] == 0:
						learning_rate = learning_rate/var_lr[1]
				
#         if(e%10 == 0 and e!=0):
# # 						save_path = saver.save(sess, checkpoint_save)
# 						print()
          
      #save all the variables
			print("creating checkpoint...")
			save_path = saver.save(sess, checkpoint_save)	
			print("checkpoint created at ", checkpoint_save)
			coord.request_stop()
			coord.join(threads)
			end_time = dt.now()
			print("total time taken =", end_time - start_time)
	return None


# **Hyper-Parameters**

In [0]:
path = "drive/share/OCR/OCR:Part3/"
data_folder = path + "dataset/tfrecords/"

max_char = 1
img_size = [128,128,1]

# digi, lc, sel uc, sel sign
checkpoint_restore = path + "checkpoints/checkpoint_digi_lc_sel_uc_sel_sign_1.ckpt"
checkpoint_save = path + "checkpoints/checkpoint_digi_lc_sel_uc_sel_sign_1.ckpt"
class_count = 56
keyword = 'digi_lc_sel_uc_sel_sign'
file_count = 1
train_data_count = 31360
batch_size = 1120
weights=[[3,3,1,16],
         [3,3,16,24],
         [3,3,24,42], 
         [3,3,42,56]]


train_filename =data_folder + 'dataset_' + keyword + '_'
test_filename = data_folder + 'dataset_' + keyword + '_'

dropout = [1, 1, 1, 1]
wd = 0.0

lr = 0.001
epochs = 10

num_of_threads=16
min_after_dequeue=10000
capacity= capacity=min_after_dequeue+(num_of_threads+1)*batch_size

# **Train**

In [0]:
train(folder_path, train_filename, test_filename, 
      train_data_count, file_count,
      weights, dropout, wd,
      img_size, max_char, class_count,
      batch_size=batch_size, learning_rate=lr, epochs=epochs, 
      restore=False, var_lr=[None,None])